In [9]:
print('在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以 Titanic_train.csv 中，首先將有遺失值的數值刪除，我們取 Titanic_train.csv 的年齡資料，試著將課堂中所學的方法應用上去。')
print('')
print("Q1：產生一個新的變數 (Age_above65_)  Age ≧ 65 為 'Y'，其餘為 'N'。")
print('')
print("Q2：將性別 (sex) 一併列入考慮，產生一個新的變數(Age_above65_female)，當 sex = female 或Age>=65為'Y'，其餘為'N'。")
print('')
print('Q3：透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate) 的相關性較高？')

在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以 Titanic_train.csv 中，首先將有遺失值的數值刪除，我們取 Titanic_train.csv 的年齡資料，試著將課堂中所學的方法應用上去。

Q1：產生一個新的變數 (Age_above65_)  Age ≧ 65 為 'Y'，其餘為 'N'。

Q2：將性別 (sex) 一併列入考慮，產生一個新的變數(Age_above65_female)，當 sex = female 或Age>=65為'Y'，其餘為'N'。

Q3：透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate) 的相關性較高？


In [11]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
%matplotlib inline

import pingouin as pg
import researchpy   
 

0.24.1


/Users/wilson/opt/anaconda3/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [12]:
# 讀入資料
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [13]:
# 產生一個新的變數 (Age_above65_)  Age ≧ 65 為 'Y'，其餘為 'N'。
# 把題目中的 Survived 用 Survived_cate 來做分析  
df_train['Survived_cate']=df_train['Survived']
df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [14]:
# 我們先把遺失值刪除
# 取出資料後，把遺失值刪除
complete_data=df_train[['Age','Survived_cate','Sex']].dropna()
display(complete_data)

,Age,Survived_cate,Sex
0,22.0,0,male
1,38.0,1,female
2,26.0,1,female
3,35.0,1,female
4,35.0,0,male
...,...,...,...
885,39.0,0,female
886,27.0,0,male
887,19.0,1,female
889,26.0,1,male


In [15]:
# Q1：產生一個新的變數 (Age_above65_)  Age ≧ 65 為 'Y'，其餘為 'N'。
def age_map1(x): 
        if(x>=65):
            return('Y')
        else: 
            return('N')
    
complete_data['Age_above65']=complete_data['Age'].apply(age_map1)

display(complete_data)

,Age,Survived_cate,Sex,Age_above65
0,22.0,0,male,N
1,38.0,1,female,N
2,26.0,1,female,N
3,35.0,1,female,N
4,35.0,0,male,N
...,...,...,...,...
885,39.0,0,female,N
886,27.0,0,male,N
887,19.0,1,female,N
889,26.0,1,male,N


In [16]:
# Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。
def age_map2(row): 
    if (row.Sex=='female'):
        return('Y')
    else: 
        if(row.Age>=65):
            return('Y')
        else:
            return('N')

complete_data['Age_above65_female']=complete_data[['Age','Sex']].apply(age_map2,axis=1)
display(complete_data)

,Age,Survived_cate,Sex,Age_above65,Age_above65_female
0,22.0,0,male,N,N
1,38.0,1,female,N,Y
2,26.0,1,female,N,Y
3,35.0,1,female,N,Y
4,35.0,0,male,N,N
...,...,...,...,...,...
885,39.0,0,female,N,Y
886,27.0,0,male,N,N
887,19.0,1,female,N,Y
889,26.0,1,male,N,N


In [20]:
# Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?
print('# Age_above65')
print('因爲離散vs離散，用Cramer’s V 係數')
print('1. 用交叉列表 pd.crosstab 整理資料')
contTable = pd.crosstab(complete_data['Age_above65'], complete_data['Survived_cate'])
contTable
print('2. 用計算自由度 df')
df = min(contTable.shape[0], contTable.shape[1]) - 1
df
print('3. 用 researchpy 計算出 Cramer’s V')
crosstab, res = researchpy.crosstab(complete_data['Survived_cate'], complete_data['Age_above65'], test='chi-square')
print("Cramer's value is",res.loc[2,'results'])
print('4. 寫判斷式輸出查表結果')
def judgment_CramerV(df,V):
    if df == 1:
        if V < 0.10:
            qual = 'negligible'
        elif V < 0.30:
            qual = 'small'
        elif V < 0.50:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 2:
        if V < 0.07:
            qual = 'negligible'
        elif V < 0.21:
            qual = 'small'
        elif V < 0.35:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 3:
        if V < 0.06:
            qual = 'negligible'
        elif V < 0.17:
            qual = 'small'
        elif V < 0.29:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 4:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.15:
            qual = 'small'
        elif V < 0.25:
            qual = 'medium'
        else:
            qual = 'large'
    else:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.13:
            qual = 'small'
        elif V < 0.22:
            qual = 'medium'
        else:
            qual = 'large'
    return(qual)

judgment_CramerV(df,res.loc[2,'results'])

# Age_above65
因爲離散vs離散，用Cramer’s V 係數
1. 用交叉列表 pd.crosstab 整理資料
2. 用計算自由度 df
3. 用 researchpy 計算出 Cramer’s V
Cramer's value is 0.0803
4. 寫判斷式輸出查表結果


'negligible'

In [21]:
print('# Age_above65_female')
print('同上作法')
contTable = pd.crosstab(complete_data['Age_above65_female'], complete_data['Survived_cate'])
contTable
df = min(contTable.shape[0], contTable.shape[1]) - 1
df
crosstab, res = researchpy.crosstab(complete_data['Survived_cate'], complete_data['Age_above65_female'], test='chi-square')
#print(res)
print("Cramer's value is",res.loc[2,'results'])


judgment_CramerV(df,res.loc[2,'results'])

# Age_above65_female
同上作法
Cramer's value is 0.514


'large'

In [22]:
'''
Age_above65_female 相關性較高
'''

'\nAge_above65_female 相關性較高\n'